In [1]:
#arcpy to process LIDAR data    

import arcpy
from arcpy import env


In [2]:
#define variables

#town name
town_name = 'Revere'

#project geodatabase
env.workspace = r'K:\DataServices\Projects\Current_Projects\Climate_Change\MVP_MMC_CoolRoofs_MVP\ArcGIS\CoolRoofs_Analysis.gdb'

env.overwriteOutput = True

#folder that contains las data
las_folder = r"K:\DataServices\Datasets\MassGIS\LiDAR\Revere pilot\LAS Data"

#dataset file path
las_dataset = r"K:\DataServices\Datasets\MassGIS\LiDAR\Revere pilot\LAS Data\Revere pilot_LasDataset.lasd"


In [ ]:
## CREATE LAS DATASET ##

point_selection_method = 'CLOSEST_TO_CENTER'


if arcpy.Exists (las_dataset): 
    arcpy.Delete_management (las_dataset)

arcpy.CreateLasDataset_management(input=las_folder,
                                  out_las_dataset=las_dataset,
                                  compute_stats='COMPUTE_STATS')
                                  

In [13]:
# BUILDING EXTRACTION # 


building_class = 6
building_layer = town_name + '_building_layer'
building_raster = town_name + '_Buildings_raster'
building_polygon_raw = town_name + '_Buildings_raw'
building_polygon_clean = town_name + '_Buildings_clean'
building_polygon_footprints = town_name + '_Building_footprints'
sampling_method = 'PREDOMINANT_CLASS'
sampling_type = 'CELLSIZE'
sampling_value = 0.5
where_clause = ' "Shape_Area" > 13.9355'

'''
inputs:
outputs:
'''

#make a las layer for building class only
lasd_layer = arcpy.management.MakeLasDatasetLayer(in_las_dataset=las_dataset, 
                                                out_layer = building_layer, 
                                                class_code=[6])

#now extract the raster
arcpy.management.LasPointStatsAsRaster(in_las_dataset="building_layer", 
                                        out_raster=building_raster, 
                                        method=sampling_method, 
                                        sampling_type=sampling_type, 
                                        sampling_value=sampling_value)

#raster to polygon
arcpy.conversion.RasterToPolygon(in_raster=building_raster, 
                                out_polygon_features=building_polygon_raw, 
                                raster_field='Value')


#only do ones larger than 150 square feet
selection = arcpy.management.SelectLayerByAttribute(in_layer_or_view=building_polygon_raw, 
                                                    where_clause=where_clause)



#eliminate noisy polygons
arcpy.management.EliminatePolygonPart(in_features=selection, 
                                        out_feature_class=building_polygon_clean, 
                                        condition='AREA',
                                        part_area=50)


#regularize building footprint
arcpy.ddd.RegularizeBuildingFootprint(in_feature=building_polygon_clean, 
                                    out_feature_class=building_polygon_footprints, 
                                    method='RIGHT_ANGLES', 
                                    tolerance=1, 
                                    densification=1, 
                                    precision=0.15, 
                                    )




<Result 'K:\\DataServices\\Projects\\Current_Projects\\Climate_Change\\MVP_MMC_CoolRoofs_MVP\\ArcGIS\\CoolRoofs_Analysis.gdb\\Revere_Buildings_raw'>

In [15]:
building_class = 6
building_layer = town_name + '_building_layer'
building_raster = town_name + '_Buildings_raster'
building_polygon_raw = town_name + '_Buildings_raw'
building_polygon_clean = town_name + '_Buildings_clean'
building_polygon_footprints = town_name + '_Building_footprints'
sampling_method = 'PREDOMINANT_CLASS'
sampling_type = 'CELLSIZE'
sampling_value = 0.5
where_clause = ' "Shape_Area" > 13.9355'

#only do ones larger than 150 square feet
selection = arcpy.management.SelectLayerByAttribute(in_layer_or_view=building_polygon_raw, 
                                                    where_clause=where_clause)



#eliminate noisy polygons
arcpy.management.EliminatePolygonPart(in_features=selection, 
                                        out_feature_class=building_polygon_clean, 
                                        condition='AREA',
                                        part_area=50)

#regularize building footprint
arcpy.ddd.RegularizeBuildingFootprint(in_features=building_polygon_clean, 
                                    out_feature_class=building_polygon_footprints, 
                                    method='RIGHT_ANGLES', 
                                    tolerance=1, 
                                    densification=1, 
                                    precision=0.15, 
                                    )

TypeError: RegularizeBuildingFootprint() got an unexpected keyword argument 'in_feature'

## Roof Extraction

Drawing from [this tutorial](https://doc.arcgis.com/en/imagery/workflows/best-practices/las-dataset-to-create-dtm-and-dsm-rasters.htm).
- Also [this video](https://www.youtube.com/watch?v=2pThNwhJ418&t=727s)

Steps:
- Generate a Digital Terrain Model (DTM) (LAS Dataset to Raster)
- Generate a Digital Surface Model (DSM) (LAS Dataset to Raster)
- Generate the Normalized Digital Surface Model (nDSM) (raster calculator)


In [8]:
## DIGITAL TERRAIN MODEL (DTM) ##

ground_layer = town_name + '_ground_layer'
ground_code = [1]
ground_return_values = ['LAST', 'LAST_OF_MANY']
out_dtm_raster = town_name + '_dtm_surface'


dtm_layer = arcpy.management.MakeLasDatasetLayer(in_las_dataset=las_dataset, 
                                                out_layer = ground_layer, 
                                                class_code = ground_code,
                                                return_values = ground_return_values)


arcpy.conversion.LasDatasetToRaster(in_las_dataset=dtm_layer, 
                                    out_raster=out_dtm_raster, 
                                    value_field='ELEVATION', 
                                    interpolation_type = 'BINNING AVERAGE LINEAR',
                                    sampling_type='CELLSIZE', 
                                    sampling_value=1.2)


## DIGITAL SURFACE MODEL (DSM) ##
surface_layer = town_name + '_surface_layer'
surface_return_values = [1, 'FIRST_OF_MANY']
out_dsm_raster = town_name + '_building_dsm_surface'
building_code = [6]


dsm_layer = arcpy.management.MakeLasDatasetLayer(in_las_dataset=las_dataset, 
                                                out_layer = surface_layer, 
                                                class_code = building_code,
                                                return_values = surface_return_values)


arcpy.conversion.LasDatasetToRaster(in_las_dataset=dsm_layer, 
                                    out_raster=out_dsm_raster, 
                                    value_field='ELEVATION',
                                    interpolation_type = 'BINNING MAXIMUM NONE',
                                    sampling_type='CELLSIZE', 
                                    sampling_value=1.2)




<Result 'K:\\DataServices\\Projects\\Current_Projects\\Climate_Change\\MVP_MMC_CoolRoofs_MVP\\ArcGIS\\CoolRoofs_Analysis.gdb\\Revere_building_dsm_surface'>

In [10]:

## NORMALIZED DIGITAL SURFACE MODEL (NDSM) ##
from arcpy.sa import *

# Set local variables
in_raster1 = out_dsm_raster
in_raster2 = out_dtm_raster
out_raster = town_name + '_ndsm_buildings'

# Excuate RasterCalculator(Minus) function
out_rc_minus_raster = RasterCalculator(rasters= [in_raster1, in_raster2], 
                                    input_names = ["x", "y"],
                                    expression="x-y")

# Save the output
out_rc_minus_raster.save(out_raster)

In [ ]:
## NDSM ##

In [ ]:
arcpy.management.BuildLasDatasetPyramid(in_las_dataset=las_dataset,
                                 point_selection_method=point_selection_method)